In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
from tensorflow.keras import backend as K


In [ ]:
# Looking at layers to pick for content and style

K.clear_session()

temp = tf.keras.applications.InceptionV3()
temp.summary()

del temp

In [12]:
# Selecting Content & Input Layers for Model

content_layers = ["conv2d_88", "conv2d_89"]
style_layers = ["conv2d", "conv2d_1", "conv2d_2","conv2d_3","conv2d_4", "conv2d_5"]

combined_layers = style_layers + content_layers

NUM_CONTENT_LAYERS = len(content_layers)
NUM_STYLE_LAYERS = len(style_layers)

print("Number of Content Layers:", len(content_layers))
print("Number of Style Layers:",  len(style_layers))

Number of Content Layers: 2
Number of Style Layers: 6


In [7]:
def inception_model(layer_names):
    
    inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False)
    
    inception.trainable = False
    
    output_layers = [inception.get_layer(name).output for name in layer_names]

    model = tf.keras.models.Model(inputs = inception.input, outputs = output_layers)

    return model

In [8]:
K.clear_session()

inception = inception_model(combined_layers)

87910968/87910968 [==============================] - 25s 0us/step


In [9]:
# Defining Loss Functions

def style_loss(features, targets):
    
    loss = tf.reduce_mean(tf.square(targets - features))

    return loss

def content_loss(features, targets):
    
    loss = 0.5 * tf.reduce_mean(tf.square(features, targets))

    return loss

def gram_matrix(input):
    
    gram = tf.linalg.einsum("bijc,bijd->bcd", input, input)
    shape = tf.shape(input)
    num_locations = tf.cast(shape[1] * shape[2], tf.float32)
    scaled_gram = gram / num_locations

    return scaled_gram